# Neural Ordinary Differential Equations 神经常微分方程

[翻译并调整自原文](https://nbviewer.org/github/urtrial/neural_ode/blob/master/Neural%20ODEs.ipynb)

**过程**很大策程度上可以用微分方程来描述:可以是物理系统的演化、病人的医疗条件、市场的基本属性等。这些数据在本质上是序列化并连续的，这意味着对它们的观测仅仅是某些不断变化的状态的具象化表示。

还有另一种类型的序列数据是离散的——例如NLP数据:它的状态是离散的，从一个符号到另一个符号，从一个单词到另一个单词。

这两种类型数据的处理通常使用循环神经网络。然而，它们在本质上是不同的，应该区别对待。

NIPS2018有一篇非常有趣的[论文](https://arxiv.org/abs/1806.07366)试图解决这个问题。作者提出了一个非常有前途的方法，他们称之为**神经常微分方程**。

在这里，我试图复制和总结原始论文的结果，使你更容易熟悉这个想法。我相信，在卷积和循环网络中，这种新的结构可能很快就会出现在很多数据科学家的工具箱中。

想象一个问题:有一个过程跟随一个未知的ODE，并随着时间有一些关于这个过程的(有噪声的)观测
$$ \frac{d\mathbf z}{d t} = f(\mathbf z(t), t) \tag{1} $$$$ \{(\mathbf z_0, t_0),(\mathbf z_1, t_1),...,(\mathbf z_M, t_M)\} - \text{observations} $$
那是否有可能找到动力学函数（dynamics function）$f(\mathbf z, t)$的近似$\widehat{f}(\mathbf z, t, \theta)$呢？

首先考虑一个稍微简单点的任务：只有两个观测量，分别在轨迹的开始和结束处，$(\mathbf z_0,t_0),(\mathbf z_N,t_N)$。系统从$
(\mathbf z_0,t_0)$开始通过使用任意ODE初值solver的参数化动态函数变化$t_N-t_0$的时间，结束时到达了新的状态$(\hat{\mathbf z_N},t_N)$，我们希望通过改变参数$\theta$来最小化$\hat{\mathbf z}_N$和观测$\mathbf z_N$的差。 

或者，更形式化地说，考虑优化以下损失函数$L(\hat{\mathbf z_N})$：
$$
L(\mathbf z(t_N)) = L \Big( \int_{t_0}^{t_N} f(\mathbf z(t), t, \theta)dt \Big) = L \big( \text{ODESolve}(\mathbf z(t_0), f, t_0, t_N, \theta) \big) \tag{2}
$$


<img src=https://raw.githubusercontent.com/ziran-dean/picbed/main/images202201062246991.png width=50% />

<p style="text-align: center"> 
    图1：梯度的连续反向传播需要及时的反向求解增广ODE。<br /> 
    箭头表示通过来自观测的梯度来调节反向传播的梯度。<br />
    图片来自论文原文
</p>

如果你不想深入研究数学，那么上面的图展示了发生的事情。黑色的轨迹表示在前向传播时求解ODE。红色的箭头在反向传播时求解伴随ODE。

为了优化$L$，我们需要计算关于参数$\mathbf z(t_0), t_0, t_N, \theta$的梯度。为了做到这点，我们需要确定loss是如何依赖于每个时刻的状态$z(t)$。
$$
\mathbf a(t) = \frac{\partial L}{\partial \mathbf z(t)} \tag{3}
$$
$a(t)$ 被称之为**“伴随”（adjoint）** ，他的动力学依据另一个ODE，它可以被认为是链式法则的瞬时模拟。

$$
\frac{d \mathbf a(t)}{d t} = -\mathbf a(t) \frac{\partial f(\mathbf z(t), t, \theta)}{\partial \mathbf z(t)} \tag{4}
$$
这个公式的推导可以在原论文的附录**B.1**中找到。

这里所有的向量都被认为是行向量，而原文中行列向量都用到了。

然后我们就可以计算：
$$
\frac{\partial L}{\partial \mathbf z(t_0)} = - \int_{t_N}^{t_0} \mathbf a(t) \frac{\partial f(\mathbf z(t), t, \theta)}{\partial \mathbf z} dt \tag{5}
$$

为了计算关于$\theta$ 和 $t$ 的梯度，我们可以把它们看作是增广状态（augmented state）的一部分，并且
$$
\frac{\partial \theta(t)}{\partial t} = \mathbf0, \frac{d t(t)}{d t} = 1,
$$
因此：
$$
\frac{d}{dt} \begin{bmatrix} \mathbf z \\ \theta \\ t \end{bmatrix} (t) = f_{\text{aug}}([\mathbf z, \theta, t]) := \begin{bmatrix} f([\mathbf z, \theta, t ]) \\ \mathbf 0 \\ 1 \end{bmatrix} \tag{6}
$$

对应的增广状态的伴随 $a_{aug}$ 为:
$$
\mathbf{a}_{\text{aug}} := \begin{bmatrix} \mathbf a \\ \mathbf a_{\theta} \\ \mathbf a_t \end{bmatrix}, \mathbf a_{\theta}(t) := \frac{\partial L}{\partial \theta(t)}, \mathbf a_t(t) := \frac{\partial L}{\partial t(t)} \tag{7}
$$

$f_{aug}$的Jacobin矩阵为：
$$
\frac{\partial f_{\text{aug}}}{\partial [\mathbf z, \theta, t]} = \begin{bmatrix} 
\frac{\partial f}{\partial \mathbf z} & \frac{\partial f}{\partial \theta} & \frac{\partial f}{\partial t} \\
\mathbf0 & \mathbf0 & \mathbf0 \\
\mathbf0 & \mathbf0 & \mathbf0
\end{bmatrix}(t) \tag{8}
$$

我们将其带入公式（4），得到：
$$
\frac{d \mathbf a_{\text{aug}}}{dt} = - \begin{bmatrix}\mathbf a(t) & \mathbf a_\theta(t) & \mathbf a_t(t)\end{bmatrix} \frac{\partial f_{\text{aug}}}{\partial [\mathbf z, \theta, t]} =- \begin{bmatrix} \mathbf a\frac{\partial f}{\partial \mathbf z} & \mathbf a\frac{\partial f}{\partial \theta} & \mathbf a\frac{\partial f}{\partial t}\end{bmatrix}(t) \tag{9}
$$

其中，第一个元素就是伴随微分方程（4），第二个元素可以用来得到对参数的**总**梯度，只需要将$\mathbf a_{\theta}(t_N) = \mathbf 0$，并且在整个区间上积分：
$$
\frac{\partial L}{\partial \theta} = \mathbf a_\theta (t_0) = - \int_{t_N}^{t_0} \mathbf a(t) \frac{\partial f(\mathbf z(t), t, \theta)}{\partial \theta} dt \tag{10}
$$

同理得到：
$$
\mathbf a_t(t_N) = \frac{d L}{d t_N} = \frac{d L}{d \mathbf z(t_N)}\frac{d \mathbf z(t_N)}{d t_N} = \mathbf a(t_N)f(\mathbf z(t_N), t_N, \theta) \tag{11}
$$

$$
\frac{\partial L}{\partial t_0} = \mathbf a_t(t_0) = \mathbf a_t(t_N) - \int_{t_N}^{t_0} \mathbf a(t) \frac{\partial f(\mathbf z(t), t, \theta)}{\partial t} dt \tag{12}
$$

对公式（5），（10）和（12）梯度的计算可以在一个ODESolver调用中计算出来：

<img src=https://raw.githubusercontent.com/ziran-dean/picbed/main/images20220111213034.png >

上面的算法描述了 ODE 初始值问题的梯度反向传播。该算法是Neural ODE的核心。
如果沿轨迹有许多观测状态，则为后续观测计算伴随增广ODE动力学，在观察时刻用直接梯度调整反向传播梯度，如上图1所示。

## 代码实现

下面的代码是Neural ODE的实现。这样做只是为了更好地了解正在发生的事情。然而，它非常接近作者[repo](https://github.com/rtqichen/torchdiffeq)中实的内容。此笔记本将理解所需的所有代码收集在一个地方，并且注释稍微多一些。对于实际使用和实验，我建议使用作者的原始实现。


In [6]:
import math
import numpy as np
from IPython.display import clear_output
from tqdm import tqdm_notebook as tqdm

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.color_palette("bright")
import matplotlib as mpl
import matplotlib.cm as cm

import torch
from torch import Tensor
from torch import nn
from torch.nn  import functional as F 
from torch.autograd import Variable

use_cuda = torch.cuda.is_available()

首先写一个任意常微分方程初始值求解器。为简单起见，我们使用欧拉ODE初值求解器，但是其他任意显式或隐式方法都可以：

In [5]:
def ode_solve(z0, t0, t1, f):
    """
    Simplest Euler ODE initial value solver
    """
    h_max = 0.05
    n_steps = math.ceil((abs(t1 - t0)/h_max).max().item())

    h = (t1 - t0)/n_steps
    t = t0
    z = z0

    for i_step in range(n_steps):
        z = z + h * f(z, t)
        t = t + h
    return z

我们通过一些方法以神经网络的形式实现了参数化动力学函数的超类。

首先，需要能够展平函数所依赖的所有参数。

其次，需要实现一种计算增广动力学的方法。这种增强的动力学取决于函数关于它的输入和参数的梯度。为了不必为每个新架构手动指定它们，我们将使用 `torch.autograd.grad` 方法

In [ ]:
class ODEF(nn.Module):
    def forward_with_grad(self, z, t, grad_outputs):
        """Compute f and a df/dz, a df/dp, a df/dt"""
        batch_size = z.shape[0]

        out = self.forward(z, t)

        a = grad_outputs
        adfdz, adfdt, *adfdp = torch.autograd.grad(
            (out,), 
            (z, t) + tuple(self.parameters()), 
            grad_outputs=(a),
            allow_unused=True, 
            retain_graph=True
        )
        # grad method automatically sums gradients for batch items, we have to expand them back 
        if adfdp is not None:
            adfdp = torch.cat([p_grad.flatten() for p_grad in adfdp]).unsqueeze(0)
            adfdp = adfdp.expand(batch_size, -1) / batch_size
        if adfdt is not None:
            adfdt = adfdt.expand(batch_size, 1) / batch_size
        return out, adfdz, adfdt, adfdp

    def flatten_parameters(self):
        p_shapes = []
        flat_parameters = []
        for p in self.parameters():
            p_shapes.append(p.size())
            flat_parameters.append(p.flatten())
        return torch.cat(flat_parameters)